In [ ]:
import pandas as pd
data_spot = pd.read_csv('/content/drive/MyDrive/Sample/네이버 지도 방문자 리뷰 크롤러_숙박_400.csv')

data_spot

In [ ]:
from glob import glob

# 1) 데이터 로드 및 컬럼 정리
file_paths = glob('/content/drive/MyDrive/Sample/네이버 지도 방문자 리뷰 크롤러_숙박_400.csv')

df = pd.concat([
    pd.read_csv(
        fp,
        usecols=[
            '가게이름','카테고리','전체평점',
            '방문자리뷰','리뷰작성자','이런_점이_좋아요','방문시간'
        ],
        engine='openpyxl'     # openpyxl 엔진 사용
    )
    for fp in file_paths
], ignore_index=True)

In [ ]:
from glob import glob

# 1) 데이터 로드 및 컬럼 정리
file_paths = glob('/content/drive/MyDrive/Sample_관광지/네이버 지도 방문자 리뷰 크롤러_관광지_*.xlsx')

df = pd.concat([
    pd.read_excel(
        fp,
        usecols=[
            '가게이름','카테고리','전체평점',
            '방문자리뷰','리뷰작성자','이런_점이_좋아요','방문시간'
        ],
        engine='openpyxl'     # openpyxl 엔진 사용
    )
    for fp in file_paths
], ignore_index=True)

# 방문시간을 문자열로 변환
df['방문시간'] = df['방문시간'].astype(str)

# 연도 포함 여부 판단: '^\d{2}\.\d{1,2}\.\d{1,2}' 형태인 경우 → 연도 있음
has_year = df['방문시간'].str.contains(r'^\d{2}\.\d{1,2}\.\d{1,2}')

# 연도 없는 데이터에는 기본 연도 '24' 추가 (또는 원하는 연도)
df.loc[~has_year, '방문시간'] = '24.' + df.loc[~has_year, '방문시간']

# 날짜 문자열만 추출
df['date_str'] = df['방문시간'].str.extract(r'(\d{2}\.\d{1,2}\.\d{1,2})')[0]

# datetime 변환
df['방문시간'] = pd.to_datetime(
    df['date_str'],
    format='%y.%m.%d',
    errors='coerce'
)

# 분기만 추출
df['quarter'] = df['방문시간'].dt.quarter

# 불필요한 중간 컬럼 제거
df.drop(columns=['date_str'], inplace=True)

#print(df[['가게이름','방문시간','quarter']].head(20))

from itertools import combinations
from collections import Counter

# 1. 필요 컬럼만 추출
df = df[['가게이름', '리뷰작성자', 'quarter']].dropna()

# 2. 사용자-분기 기준 그룹핑하여 장소 목록 생성
user_quarter_group = df.groupby(['리뷰작성자', 'quarter'])['가게이름'].apply(list)

# 3. 각 그룹에서 장소 쌍 조합 생성
co_review_pairs = []

for places in user_quarter_group:
    # 장소가 2개 이상인 경우만 조합
    if len(set(places)) >= 2:
        pairs = combinations(sorted(set(places)), 2)
        co_review_pairs.extend(pairs)

# 4. 장소 쌍별 등장 횟수 계산
pair_counts = Counter(co_review_pairs)

# 5. 결과를 DataFrame으로 변환
pair_df = pd.DataFrame([
    {'가게1': p1, '가게2': p2, 'co_review_count': count}
    for (p1, p2), count in pair_counts.items()
])

# 6. co_review 수 내림차순 정렬
pair_df = pair_df.sort_values(by='co_review_count', ascending=False)

# 7. 결과 확인
print(pair_df.head(20))

# 8. 저장 (선택사항)
pair_df.to_csv('/content/drive/MyDrive/Sample/관광지_co_review_pairs.csv', index=False)


In [ ]:
datar = pd.read_csv('/content/drive/MyDrive/Sample/관광지_co_review_pairs.csv')

datar